In [ ]:
import src.data.utilities as utils
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
## DATASETS ##
# 1. Sentencing data 2017–21 (from: https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1076592/Data-behind-interactive-tools-3.zip)
cols = ['Police Force Area', 'Year', 'Sex', 'Age group', 'Offence group', 'Sentence Outcome', 'Custodial Sentence Length','Sentenced']
df = utils.loadData("data/external/sentencing.csv", cols=cols)
df.head()

In [ ]:
# 2. Court outcomes by police force area 2009–2019 (from: https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/888561/csvs-behind-data-tools-2-2019.zip)
cols_2009 =['Police Force Area', 'Year of Appearance', 'Sex', 'Age Group', 'Offence Group', 'Outcome', 'Custodial Sentence Length','Count'] 
df_2009 = utils.loadData('data/external/court-outcomes-by-PFA-2019.csv', cols=cols_2009)
df_2009.head()

In [ ]:
#Dropping duplicate data from 2009 dataset that also appears in df
filt = df_2009['Year of Appearance'] < 2017
df_2009 = df_2009[filt].copy()

In [ ]:
df_list = utils.dataframeList(locals())

In [ ]:
#Standardising variable names
for data in df_list:
    utils.lcColumns(data)
    utils.renameColumns(data, columns={
        'year_of_appearance': 'year',
        'offence_group': 'offence',
        'police_force_area': 'pfa',
        'sentence_outcome': 'outcome',
        'custodial_sentence_length': 'sentence_length',
        'sentenced': 'freq',
        'count': 'freq'}
        )
    # utils.orderColumns(data, column_order = ['year', 'pfa', 'sex', 'age_group', 'offence', 'outcome', 'sentence_length', 'freq'])

In [ ]:

def tidy_elements(data):
    regex = {r"^\S*: \S* - ": "",
             r"\d\d: ": "",
            "Total ": "",
            "(Over)": "More than",
            "( and including)": "",
            "(to less than)": "and under",
            "Life$": "Life sentence"
             }
    return data.map(regex)

In [ ]:
df_combined = pd.concat(df_list)
tidy_elements(df_combined)

In [ ]:
df_combined = pd.concat(df_list)
utils.tidy_elements(df_combined)

In [ ]:
df_combined.dtypes

In [ ]:
convert_dict = {'outcome': "category",
                'sentence_length': "category"
                }

In [ ]:
df_combined = df_combined.astype(convert_dict)

In [ ]:
df_combined.dtypes

In [ ]:
df = pd.read_csv('data/interim/PFA_2009-21_women_cust_comm_sus.csv')

In [ ]:
filt = df['outcome'] == 'Immediate custody'
pfa_custody_sentence_lengths = df[filt].copy()

In [ ]:
pfa_custody_sentence_lengths

Defining sentence length categories

In [ ]:
less_6months = ["Up to 1 month", 
                "More than 1 month and up to 2 months",
                "More than 2 months and up to 3 months",
                "More than 3 months and under 6 months"]

six_12_months = ["6 months",
                "More than 6 months and up to 9 months",
                "More than 9 months and under 12 months"]

In [ ]:
def sentence_length_groups(sentence_len):
    if sentence_len in less_6months:
        return 'Less than 6 months'
    elif sentence_len in six_12_months:
        return '6 months and under 12 months'
    else:
        return 'Over 12 months'

In [ ]:
def replace_sentence_lengths(x_df, fill_map):
    res=x_df.loc[:,'sentence_len'].map(fill_map)
    x_df.loc[:,'sentence_len']=res 
    
    return x_df

In [ ]:
(
    df
    .query('outcome == @sentence_type')
    .pipe(replace_sentence_lengths, sentence_length_groups)
    
)

In [ ]:
#Filtering for custodial sentences and applying the map

pfa_custody_sentence_lengths = df.query('outcome == @sentence_type')
pfa_custody_sentence_lengths['sentence_len'] = pfa_custody_sentence_lengths['sentence_len'].map(sentence_length_groups)

#Grouping dataset
pfa_custody_sentence_lengths = pfa_custody_sentence_lengths.groupby(['pfa', 'year', 'sentence_len'], as_index=False)['freq'].sum()

#Outputting to CSV
pfa_custody_sentence_lengths.to_csv('data/interim/PFA_2009-21_women_cust_sentence_len.csv', index=False)

In [ ]:
df2 = pfa_custody_sentence_lengths.copy()

In [ ]:
df2['sentence_len'] = df2['sentence_len'].map(sentence_length_groups)

In [ ]:
df2['sentence_len'].value_counts()

In [ ]:
#Importing cleansed dataset
df = pd.read_csv('data/interim/PFA_2009-21_women_cust_comm_sus.csv')

## 1.SENTENCING OUTCOME FOR EACH PFA BY YEAR

#Grouping dataset
pfa_sentencing_outcomes = df.groupby(['pfa', 'year', 'outcome'], as_index=False)['freq'].sum()

#Outputting to CSV
pfa_sentencing_outcomes.to_csv('data/processed/PFA_2009-21_women_sentencing_outcomes_FINAL.csv', index=False)


## 2.CUSTODIAL SENTENCE LENGTHS FOR EACH PFA BY YEAR
'''THIS PRODUCES THE DATA FOR FIGURE 1 IN THE PFA FACTSHEET'''

#Filtering cleansed dataset
filt = df['outcome'] == 'Immediate custody'
pfa_custody_sentence_lengths = df[filt].copy()

#Defining sentence_len categories
less_6months = ["Up to 1 month", 
                "More than 1 month and up to 2 months",
                "More than 2 months and up to 3 months",
                "More than 3 months and under 6 months"]

six_12_months = ["6 months",
                "More than 6 months and up to 9 months",
                "More than 9 months and under 12 months"]

#Mapping sentence_len categories
def sentence_length_groups(sentence_len):
    if sentence_len in less_6months:
        return 'Less than 6 months'
    elif sentence_len in six_12_months:
        return '6 months and under 12 months'
    else:
        return 'Over 12 months'
    
pfa_custody_sentence_lengths['sentence_len'] = pfa_custody_sentence_lengths['sentence_len'].map(sentence_length_groups)

#Grouping dataset
pfa_custody_sentence_lengths = pfa_custody_sentence_lengths.groupby(['pfa', 'year', 'sentence_len'], as_index=False)['freq'].sum()

#Outputting to CSV
# final_df.to_csv('data/interim/PFA_2009-21_women_cust_sentence_len_test.csv', index=False)

In [ ]:
#By year
filt = pfa_custody_sentence_lengths['year'] >= 2014
pfa_df_2014 = pfa_custody_sentence_lengths[filt].copy()

#By sentences of less than six months
filt = pfa_df_2014['sentence_len'] == "Less than 6 months"
lt_6 = pfa_df_2014[filt].copy()

#By sentences of less than 12 months
filt = pfa_df_2014['sentence_len'] != "Over 12 months"
lt_12m = pfa_df_2014[filt].copy()

In [ ]:
def aggregate_sentences(df):
    new_df = pd.crosstab(index=df['pfa'], columns=df['year'],
                        values=df['freq'], aggfunc='sum')
    
    new_df = new_df.fillna(0.0).astype(int)
    new_df['per_change_2014'] = new_df.pct_change(axis='columns', periods=7).dropna(axis='columns')
    return new_df

In [ ]:
lt_12m.columns

In [ ]:
aggregate_sentences(lt_12m)

In [ ]:
df = pd.read_csv('data/interim/PFA_2009-21_women_cust_comm_sus.csv')

In [ ]:
(
    df
    .groupby(['pfa', 'year', 'outcome'], as_index=False)['freq'].sum()
    .to_csv('data/processed/PFA_2009-21_women_sentencing_outcomes_TEST.csv', index=False)
)

In [ ]:
sentence_type = 'Immediate custody'
year = 2021

In [ ]:
def crosstab(index, columns, values=None, aggfunc=None):
    return pd.crosstab(index, columns, values=values, aggfunc=aggfunc)

### Following publication of new dataset, testing existing `data_processing.py` script logic to ensure QA

In [ ]:
import src.data.utilities as utils
%load_ext autoreload
%autoreload 2

import pandas as pd

In [ ]:
#Importing cleansed dataset
df = pd.read_csv('data/interim/PFA_2009-22_women_cust_comm_sus.csv')
df.head()

In [ ]:
## 1.SENTENCING OUTCOME FOR EACH PFA BY YEAR

#Grouping dataset and saving out
(
    df
    .groupby(['pfa', 'year', 'outcome'], as_index=False)['freq'].sum()
    .to_csv('data/processed/PFA_2009-22_women_sentencing_outcomes_FINAL.csv', index=False)
)

In [ ]:
## 2. CUSTODIAL SENTENCES FOR EACH PFA BY OFFENCE TYPE

#Filtering cleansed dataset and grouping by PFA and offence group 
sentence_type = 'Immediate custody'
year = 2022

pfa_custody = (
    df
    .query("outcome == @sentence_type & year == @year")
    .groupby(['pfa', 'offence'], as_index=False)['freq'].sum()
)

#Using crosstab with normalize argument to calculate offence group proportions by PFA
pfa_custody_offences = pd.crosstab(index=pfa_custody['pfa'], columns=pfa_custody['offence'], values=pfa_custody['freq'], aggfunc=sum, normalize='index').round(3)

#Outputting to CSV
pfa_custody_offences.to_csv('data/processed/PFA_2022_offences.csv', index=False)

In [ ]:
df['sentence_length'].unique()

In [ ]:
## 3.CUSTODIAL SENTENCE LENGTHS FOR EACH PFA BY YEAR
'''THIS PRODUCES THE DATA FOR FIGURE 1 IN THE PFA FACTSHEET'''

#Defining sentence_len categories
less_6months = ["Up to 1 month", 
                "More than 1 month and up to 2 months",
                "More than 2 months and up to 3 months",
                "More than 3 months and under 6 months"]

six_12_months = ["6 months",
                "More than 6 months and up to 9 months",
                "More than 9 months and under 12 months"]

#Mapping sentence_len categories
def sentence_length_groups(sentence_len):
    if sentence_len in less_6months:
        return 'Less than 6 months'
    elif sentence_len in six_12_months:
        return '6 months and under 12 months'
    else:
        return 'Over 12 months'

#Filtering for custodial sentences and applying the map

pfa_custody_sentence_lengths = df.query('outcome == @sentence_type').copy()
pfa_custody_sentence_lengths['sentence_length'] = pfa_custody_sentence_lengths['sentence_length'].map(sentence_length_groups)

#Grouping dataset
pfa_custody_sentence_lengths = pfa_custody_sentence_lengths.groupby(['pfa', 'year', 'sentence_length'], as_index=False)['freq'].sum()

#Outputting to CSV
pfa_custody_sentence_lengths.to_csv('data/interim/PFA_2009-22_women_cust_sentence_length.csv', index=False)

In [ ]:
## 4. CUSTODIAL SENTENCES FOR EACH PFA BY YEAR

'''THIS PRODUCES THREE DATASETS: 
    * TOTAL NUMBER OF WOMEN SENTENCED TO CUSTODY BY PFA; AND OF THOSE 
        * SENTENCED TO LESS THAN SIX MONTHS; AND
        * SENTENCED TO LESS THAN 12 MONTHS'''

#FILTERING DATA

#By year
filt = pfa_custody_sentence_lengths['year'] >= 2014
pfa_df_2014 = pfa_custody_sentence_lengths[filt].copy()

#By sentences of less than six months
filt = pfa_df_2014['sentence_length'] == "Less than 6 months"
lt_6m = pfa_df_2014[filt].copy()

#By sentences of less than 12 months
filt = pfa_df_2014['sentence_length'] != "Over 12 months"
lt_12m = pfa_df_2014[filt].copy()

#Defining new function for aggregating data and adding a percentage change column
def aggregate_sentences(df):
    new_df = pd.crosstab(index=df['pfa'], columns=df['year'],
                        values=df['freq'], aggfunc='sum')
    
    new_df = new_df.fillna(0.0).astype(int)
    new_df['per_change_2014'] = new_df.pct_change(axis='columns', periods=8).dropna(axis='columns')
    return new_df

#Using dictionary comprehension to run both DataFrames through the function
'''This returns a new dictionary df_dict with _table added to the keys. Values can be accessed using df_dict['key'] and DataFrame functionality is retained
See https://stackoverflow.com/questions/51845732/apply-a-function-to-multiple-dataframes-return-multiple-dfs-as-output'''

sentence_length_dict = {'cust_sentences_total':pfa_df_2014, 'cust_sentences_lt_6m':lt_6m, 'cust_sentences_lt_12m': lt_12m}
df_dict = {i+'_table': aggregate_sentences(sentence_length) for i, sentence_length in sentence_length_dict.items()}

#Outputting to CSV
#These are the final versions ready for formatting and publication
for i, df in df_dict.items():
    df.to_csv(f'data/processed/UPDATED{i}.csv')

Check the output of these tables for consistency against earlier dataset.

In [ ]:
updated_df = pd.read_csv("data/processed/UPDATEDcust_sentences_total_table.csv")
updated_df

In [ ]:
original_df = pd.read_csv("data/processed/cust_sentences_total_table.csv")
original_df

In [ ]:
drop = ['2022', 'per_change_2014']
updated_df.drop(drop, axis=1, inplace=True)
original_df.drop(['per_change_2014'], axis=1, inplace=True)

In [ ]:
updated_df.compare(original_df, align_axis=1)

Just a difference of one in only two PFA 2017 cases. As before I suspect this is simply a data revision, the difference is so minor.

Let's also compare the sentence length data produced at stage 3

In [ ]:
original_df = pd.read_csv("data/interim/PFA_2009-21_women_cust_sentence_len.csv")
original_df

In [ ]:
updated_df = pd.read_csv("data/interim/PFA_2009-22_women_cust_sentence_length.csv")
updated_df

Filter out the 2022 values from `updated_df`

In [ ]:
filt = updated_df['year'] != 2022
updated_df = updated_df[filt].copy()
updated_df = updated_df.reset_index(drop=True)

Renaming the `sentence_len` column in `original_df`

In [ ]:
original_df.rename(columns={'sentence_len': "sentence_length"}, inplace=True)

In [ ]:
original_df

In [ ]:
original_df.sort_index(inplace=True)
updated_df.sort_index(inplace=True)

In [ ]:
original_df

In [ ]:
updated_df

In [ ]:
original_df.compare(updated_df)

Starting to get a bit confused here. There appear to be two additional rows in the `updated_df` which I suspect are what is causing this error.

In [ ]:
print(original_df.columns, updated_df.columns)

In [ ]:
updated_df['sentence_length'].unique()

In [ ]:
updated_count = pd.crosstab(index=updated_df['pfa'], columns=['year'])

In [ ]:
original_count = pd.crosstab(index=original_df['pfa'], columns=['year'])

Let's now compare these two crosstabs

In [ ]:
original_count.compare(updated_count, align_axis=1)

Right, so there are a few differences here. Let's check these out further.

First there's one fewer entry in the `updated_df` for Cleveland

In [ ]:
original_df.query('pfa == "Cleveland"').groupby(['year']).size() == updated_df.query('pfa == "Cleveland"').groupby(['year']).size()

Right, that appears to be in 2014, which is a little odd. Let's see what the difference is.

In [ ]:
pfa = 'Cleveland'
year = 2014
for df in [original_df, updated_df]:
    print(df.query('pfa == @pfa & year == @year'))

Whilst at first glance the difference in values is a bit concerning. It's worth noting that the totals are broadly the same 90 vs 91. Let's dig into this a bit further by revisiting some earlier versions of this data.

Starting by looking at some of the `nan` and `24:not known` entries within `sentence_length`

In [ ]:
df['sentence_length'].unique()

In [ ]:
sentence_type = 'Immediate custody'
df.query('outcome == @sentence_type & sentence_length == "24:not known"')

In [ ]:
df.query('outcome == @sentence_type & sentence_length.isnull()')

Okay, well it's not that. Let's dig into the count of the unconsolidated sentence lengths.

In [ ]:
cleveland = df.query('pfa == "Cleveland" & outcome == @sentence_type').copy()
cleveland

Dropping 2022 data

In [ ]:
cleveland = df.query('pfa == "Cleveland" & outcome == @sentence_type & year < 2022').copy()
cleveland

In [ ]:
cleveland.groupby(['year', 'sentence_length'], as_index=False)['freq'].sum()

In [ ]:
#Reading in 2021 equivalent dataset
df_2021 = pd.read_csv('data/interim/PFA_2009-21_women_cust_comm_sus.csv')
df_2021

In [ ]:
cleveland_2021 = df_2021.query('pfa == "Cleveland" & outcome == @sentence_type').copy()
cleveland_2021

In [ ]:
cleveland_2021.groupby(['year'])['freq'].sum()

In [ ]:
cleveland.groupby(['year'])['freq'].sum()

In [ ]:
cleveland_2021.groupby(['sentence_len'])['freq'].sum()

In [ ]:
cleveland.groupby(['sentence_length'])['freq'].sum()

Aha, there's some weirdness going on with the `sentence_lengths`. There are some which say they are "under" and others that say they are "up to". Let's see which years include these.

In [ ]:
cleveland.query('sentence_length == "More than 3 months and up to 6 months"')['year'].unique()

I think this is a newly introduced issue by wording change in the 2010 data onwards. Let's circle back to the importation of the original datasets and see whether the issue lies there.

In [ ]:
import src.data.utilities as utils
%load_ext autoreload
%autoreload 2

import pandas as pd
import glob

In [ ]:
path="data/external/obo_sent_pivot_2010_2022/"
all_files = glob.glob(path + "*.csv")


## IMPORTING DATASETS ##
# 1. Sentencing data 2010–22 
# (from: https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1157979/obo_sent_pivot_2010_2015.zip and 
# https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1157991/obo_sent_pivot_2016_2022.zip)

cols = ['Police Force Area', 'Year', 'Sex', 'Age group', 'Offence group', 'Sentence Outcome', 'Custodial Sentence Length','Sentenced']
all_csvs = [utils.loadData(filename, cols=cols) for filename in all_files]
df = pd.concat(all_csvs, axis=0, ignore_index=True)

In [ ]:
sorted(df['Custodial Sentence Length'].unique())

Right, there is a clear difference in the naming convention here, "and including" is specific and this is missing in the 6 month, 12 month and 4 years entries. Let's check that my regex work isn't causing the issue.

Okay, let's perform the column transformations first to make filtering the dataset more straightforward

In [ ]:
# Renaming columns
utils.lcColumns(df)
utils.renameColumns(df, columns={
    'year_of_appearance': 'year',
    'offence_group': 'offence',
    'police_force_area': 'pfa',
    'sentence_outcome': 'outcome',
    'custodial_sentence_length': 'sentence_length',
    'sentenced': 'freq',
    'count': 'freq'}
    )

Now using my `tidy_elements()` function to double check the impact.

In [ ]:
utils.tidy_elements(df)

In [ ]:
sorted(df['sentence_length'].unique())

Right, so my regex is losing that subtlety for those three groups, so I need to adjust this.

Done, and re-run the above code with that element of the regex removed. Now to switch back to `2022_data_match_testing.ipynb` to see if I can output the data as expected.

Right, `PFA_2010-22_women_cust_comm_sus.csv` has now been output. Let's attempt this again.

In [1]:
import src.data.utilities as utils
%load_ext autoreload
%autoreload 2

import pandas as pd

In [2]:
#Importing cleansed dataset
df = pd.read_csv('data/interim/PFA_2010-22_women_cust_comm_sus.csv')
df.head()

,year,pfa,sex,age_group,offence,outcome,sentence_length,freq
0,2010,Avon and Somerset,Female,Young adults,Violence against the person,Community sentence,24:Not known,1
1,2010,Avon and Somerset,Female,Young adults,Drug offences,Community sentence,24:Not known,1
2,2010,Avon and Somerset,Female,Young adults,Violence against the person,Immediate custody,Life sentence,1
3,2010,Avon and Somerset,Female,Young adults,Violence against the person,Community sentence,24:Not known,1
4,2010,Avon and Somerset,Female,Young adults,Violence against the person,Suspended sentence,24:Not known,1


In [3]:
## 1.SENTENCING OUTCOME FOR EACH PFA BY YEAR

#Grouping dataset and saving out
(
    df
    .groupby(['pfa', 'year', 'outcome'], as_index=False)['freq'].sum()
    .to_csv('data/processed/PFA_2010-22_women_sentencing_outcomes_FINAL.csv', index=False)
)

In [6]:
df1 = df.groupby(['pfa', 'year', 'outcome'], as_index=False)['freq'].sum()
df1.head()

,pfa,year,outcome,freq
0,Avon and Somerset,2010,Community sentence,618
1,Avon and Somerset,2010,Immediate custody,173
2,Avon and Somerset,2010,Suspended sentence,164
3,Avon and Somerset,2011,Community sentence,732
4,Avon and Somerset,2011,Immediate custody,206


In [7]:
df1.query('pfa == "Cleveland" & 2020 > year > 2015')

,pfa,year,outcome,freq
174,Cleveland,2016,Community sentence,341
175,Cleveland,2016,Immediate custody,108
176,Cleveland,2016,Suspended sentence,175
177,Cleveland,2017,Community sentence,335
178,Cleveland,2017,Immediate custody,152
179,Cleveland,2017,Suspended sentence,183
180,Cleveland,2018,Community sentence,368
181,Cleveland,2018,Immediate custody,140
182,Cleveland,2018,Suspended sentence,125
183,Cleveland,2019,Community sentence,306


Checked these values against previously published edition of Cleveland factsheet and the data is spot on. 

Let's move on to the next output.

In [8]:
## 2. CUSTODIAL SENTENCES FOR EACH PFA BY OFFENCE TYPE

#Filtering cleansed dataset and grouping by PFA and offence group 
sentence_type = 'Immediate custody'
year = 2022

pfa_custody = (
    df
    .query("outcome == @sentence_type & year == @year")
    .groupby(['pfa', 'offence'], as_index=False)['freq'].sum()
)

#Using crosstab with normalize argument to calculate offence group proportions by PFA
pfa_custody_offences = pd.crosstab(index=pfa_custody['pfa'], columns=pfa_custody['offence'], values=pfa_custody['freq'], aggfunc=sum, normalize='index').round(3)

In [9]:
pfa_custody_offences

offence,Criminal damage and arson,Drug offences,Fraud Offences,Miscellaneous crimes against society,Possession of weapons,Public order offences,Robbery,Sexual offences,Summary motoring,Summary non-motoring,Theft offences,Violence against the person
pfa,,,,,,,,,,,,
Avon and Somerset,0.017,0.103,0.017,0.026,0.009,0.069,0.009,0.009,0.017,0.155,0.224,0.345
Bedfordshire,0.000,0.079,0.000,0.053,0.132,0.053,0.026,0.026,0.000,0.079,0.316,0.237
Cambridgeshire,0.000,0.118,0.044,0.015,0.059,0.074,0.015,0.000,0.000,0.162,0.324,0.191
Cheshire,0.014,0.122,0.108,0.014,0.041,0.054,0.027,0.000,0.014,0.095,0.297,0.216
Cleveland,0.010,0.080,0.030,0.060,0.060,0.020,0.050,0.020,0.030,0.070,0.410,0.160
Cumbria,0.000,0.069,0.000,0.000,0.000,0.172,0.000,0.000,0.034,0.069,0.345,0.310
Derbyshire,0.000,0.057,0.057,0.025,0.025,0.041,0.025,0.000,0.033,0.082,0.443,0.213
Devon and Cornwall,0.000,0.032,0.032,0.032,0.111,0.159,0.000,0.048,0.063,0.095,0.222,0.206
Dorset,0.043,0.174,0.130,0.043,0.043,0.000,0.000,0.000,0.000,0.087,0.174,0.304


In [ ]:
#Outputting to CSV
pfa_custody_offences.to_csv('data/processed/PFA_2022_offences.csv', index=False)

In [ ]:
df['sentence_length'].unique()

Updating the sentence_length categories with the new wording, after changing my `tidy_elements()` function

In [24]:
## 3.CUSTODIAL SENTENCE LENGTHS FOR EACH PFA BY YEAR
'''THIS PRODUCES THE DATA FOR FIGURE 1 IN THE PFA FACTSHEET'''

#Defining sentence_length categories—THESE HAVE BEEN UPDATED
less_6months = ["Up to and including 1 month", 
                "More than 1 month and up to and including 2 months",
                "More than 2 months and up to and including 3 months",
                "More than 3 months and up to 6 months"]

six_12_months = ["6 months",
                "More than 6 months and up to and including 9 months",
                "More than 9 months and up to 12 months"]

#Mapping sentence_len categories
def sentence_length_groups(sentence_len):
    if sentence_len in less_6months:
        return 'Less than 6 months'
    elif sentence_len in six_12_months:
        return '6 months and under 12 months'
    else:
        return 'Over 12 months'

#Filtering for custodial sentences and applying the map

pfa_custody_sentence_lengths = df.query('outcome == @sentence_type').copy()
pfa_custody_sentence_lengths['sentence_length'] = pfa_custody_sentence_lengths['sentence_length'].map(sentence_length_groups)

#Grouping dataset
pfa_custody_sentence_lengths = pfa_custody_sentence_lengths.groupby(['pfa', 'year', 'sentence_length'], as_index=False)['freq'].sum()

In [25]:
pfa_custody_sentence_lengths

,pfa,year,sentence_length,freq
0,Avon and Somerset,2010,6 months and under 12 months,16
1,Avon and Somerset,2010,Less than 6 months,113
2,Avon and Somerset,2010,Over 12 months,44
3,Avon and Somerset,2011,6 months and under 12 months,21
4,Avon and Somerset,2011,Less than 6 months,142
...,...,...,...,...
1623,Wiltshire,2021,Less than 6 months,15
1624,Wiltshire,2021,Over 12 months,15
1625,Wiltshire,2022,6 months and under 12 months,5
1626,Wiltshire,2022,Less than 6 months,18


In [13]:
original_df = pd.read_csv("data/interim/PFA_2009-21_women_cust_sentence_len.csv")
original_df

,pfa,year,sentence_len,freq
0,Avon and Somerset,2009,6 months and under 12 months,11
1,Avon and Somerset,2009,Less than 6 months,117
2,Avon and Somerset,2009,Over 12 months,37
3,Avon and Somerset,2010,6 months and under 12 months,16
4,Avon and Somerset,2010,Less than 6 months,113
...,...,...,...,...
1626,Wiltshire,2020,Less than 6 months,15
1627,Wiltshire,2020,Over 12 months,13
1628,Wiltshire,2021,6 months and under 12 months,3
1629,Wiltshire,2021,Less than 6 months,15


Let's drop 2009 figures and see if we can `compare()` the two dataframes

In [14]:
original_df = original_df.query('year != 2009')
original_df

,pfa,year,sentence_len,freq
3,Avon and Somerset,2010,6 months and under 12 months,16
4,Avon and Somerset,2010,Less than 6 months,113
5,Avon and Somerset,2010,Over 12 months,44
6,Avon and Somerset,2011,6 months and under 12 months,21
7,Avon and Somerset,2011,Less than 6 months,142
...,...,...,...,...
1626,Wiltshire,2020,Less than 6 months,15
1627,Wiltshire,2020,Over 12 months,13
1628,Wiltshire,2021,6 months and under 12 months,3
1629,Wiltshire,2021,Less than 6 months,15


And let's drop 2022 data in `pfa_custody_sentence_lengths`

In [26]:
pfa_custody_sentence_lengths = pfa_custody_sentence_lengths.query('year != 2022')
pfa_custody_sentence_lengths

,pfa,year,sentence_length,freq
0,Avon and Somerset,2010,6 months and under 12 months,16
1,Avon and Somerset,2010,Less than 6 months,113
2,Avon and Somerset,2010,Over 12 months,44
3,Avon and Somerset,2011,6 months and under 12 months,21
4,Avon and Somerset,2011,Less than 6 months,142
...,...,...,...,...
1620,Wiltshire,2020,Less than 6 months,15
1621,Wiltshire,2020,Over 12 months,13
1622,Wiltshire,2021,6 months and under 12 months,3
1623,Wiltshire,2021,Less than 6 months,15


Okay, this is a good start, for the first time I've got the same number of rows. Let's investigate whether we can `compare()`

In [16]:
original_df.rename(columns={'sentence_len': "sentence_length"}, inplace=True)

/var/folders/kp/698d00016j9g3g0cyrwpwpw00000gn/T/ipykernel_76963/1788724509.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  original_df.rename(columns={'sentence_len': "sentence_length"}, inplace=True)


In [29]:
original_df = original_df.reset_index(drop=True)
pfa_custody_sentence_lengths = pfa_custody_sentence_lengths.reset_index(drop=True)

In [30]:
original_df

,pfa,year,sentence_length,freq
0,Avon and Somerset,2010,6 months and under 12 months,16
1,Avon and Somerset,2010,Less than 6 months,113
2,Avon and Somerset,2010,Over 12 months,44
3,Avon and Somerset,2011,6 months and under 12 months,21
4,Avon and Somerset,2011,Less than 6 months,142
...,...,...,...,...
1500,Wiltshire,2020,Less than 6 months,15
1501,Wiltshire,2020,Over 12 months,13
1502,Wiltshire,2021,6 months and under 12 months,3
1503,Wiltshire,2021,Less than 6 months,15


In [31]:
pfa_custody_sentence_lengths

,pfa,year,sentence_length,freq
0,Avon and Somerset,2010,6 months and under 12 months,16
1,Avon and Somerset,2010,Less than 6 months,113
2,Avon and Somerset,2010,Over 12 months,44
3,Avon and Somerset,2011,6 months and under 12 months,21
4,Avon and Somerset,2011,Less than 6 months,142
...,...,...,...,...
1500,Wiltshire,2020,Less than 6 months,15
1501,Wiltshire,2020,Over 12 months,13
1502,Wiltshire,2021,6 months and under 12 months,3
1503,Wiltshire,2021,Less than 6 months,15


In [32]:
original_df.compare(pfa_custody_sentence_lengths, align_axis=1)

freq      
     self other
275  43.0  44.0
810  46.0  47.0

### FINALLY
We have been able to compare and it appears as though there are two additional values in the new dataframe—which matches up with the two additional values in the earlier dataframe `PFA_2010-22_women_cust_comm_sus.csv`. Let's just double check these indexes.

In [42]:
pfa_custody_sentence_lengths.iloc[[275, 810]]

,pfa,year,sentence_length,freq
275,Devon and Cornwall,2017,Over 12 months,44
810,Merseyside,2017,Over 12 months,47


In [43]:
original_df.iloc[[275, 810]]

,pfa,year,sentence_length,freq
275,Devon and Cornwall,2017,Over 12 months,43
810,Merseyside,2017,Over 12 months,46


Great, let's re-run section three with the csv output

In [44]:
## 3.CUSTODIAL SENTENCE LENGTHS FOR EACH PFA BY YEAR
'''THIS PRODUCES THE DATA FOR FIGURE 1 IN THE PFA FACTSHEET'''

#Defining sentence_length categories—THESE HAVE BEEN UPDATED
less_6months = ["Up to and including 1 month", 
                "More than 1 month and up to and including 2 months",
                "More than 2 months and up to and including 3 months",
                "More than 3 months and up to 6 months"]

six_12_months = ["6 months",
                "More than 6 months and up to and including 9 months",
                "More than 9 months and up to 12 months"]

#Mapping sentence_len categories
def sentence_length_groups(sentence_len):
    if sentence_len in less_6months:
        return 'Less than 6 months'
    elif sentence_len in six_12_months:
        return '6 months and under 12 months'
    else:
        return 'Over 12 months'

#Filtering for custodial sentences and applying the map

pfa_custody_sentence_lengths = df.query('outcome == @sentence_type').copy()
pfa_custody_sentence_lengths['sentence_length'] = pfa_custody_sentence_lengths['sentence_length'].map(sentence_length_groups)

#Grouping dataset
pfa_custody_sentence_lengths = pfa_custody_sentence_lengths.groupby(['pfa', 'year', 'sentence_length'], as_index=False)['freq'].sum()

#Outputting to CSV
pfa_custody_sentence_lengths.to_csv('data/interim/PFA_2010-22_women_cust_sentence_length.csv', index=False)

In [45]:
## 4. CUSTODIAL SENTENCES FOR EACH PFA BY YEAR

'''THIS PRODUCES THREE DATASETS: 
    * TOTAL NUMBER OF WOMEN SENTENCED TO CUSTODY BY PFA; AND OF THOSE 
        * SENTENCED TO LESS THAN SIX MONTHS; AND
        * SENTENCED TO LESS THAN 12 MONTHS'''

#FILTERING DATA

#By year
filt = pfa_custody_sentence_lengths['year'] >= 2014
pfa_df_2014 = pfa_custody_sentence_lengths[filt].copy()

#By sentences of less than six months
filt = pfa_df_2014['sentence_length'] == "Less than 6 months"
lt_6m = pfa_df_2014[filt].copy()

#By sentences of less than 12 months
filt = pfa_df_2014['sentence_length'] != "Over 12 months"
lt_12m = pfa_df_2014[filt].copy()

#Defining new function for aggregating data and adding a percentage change column
def aggregate_sentences(df):
    new_df = pd.crosstab(index=df['pfa'], columns=df['year'],
                        values=df['freq'], aggfunc='sum')
    
    new_df = new_df.fillna(0.0).astype(int)
    new_df['per_change_2014'] = new_df.pct_change(axis='columns', periods=8).dropna(axis='columns')
    return new_df

#Using dictionary comprehension to run both DataFrames through the function
'''This returns a new dictionary df_dict with _table added to the keys. Values can be accessed using df_dict['key'] and DataFrame functionality is retained
See https://stackoverflow.com/questions/51845732/apply-a-function-to-multiple-dataframes-return-multiple-dfs-as-output'''

sentence_length_dict = {'cust_sentences_total':pfa_df_2014, 'cust_sentences_lt_6m':lt_6m, 'cust_sentences_lt_12m': lt_12m}
df_dict = {i+'_table': aggregate_sentences(sentence_length) for i, sentence_length in sentence_length_dict.items()}

#Outputting to CSV
#These are the final versions ready for formatting and publication
for i, df in df_dict.items():
    df.to_csv(f'data/processed/2022_{i}.csv')

Now to compare these 2022 tables with the original tables

In [46]:
new_total_table = pd.read_csv('data/processed/2022_cust_sentences_total_table.csv')
new_total_table

,pfa,2014,2015,2016,2017,2018,2019,2020,2021,2022,per_change_2014
0,Avon and Somerset,196,165,164,158,148,151,103,103,116,-0.408163
1,Bedfordshire,69,80,53,53,36,31,23,20,38,-0.449275
2,Cambridgeshire,91,89,112,115,116,89,78,47,68,-0.252747
3,Cheshire,169,181,167,172,176,149,123,117,74,-0.562130
4,Cleveland,91,78,108,152,140,98,55,103,100,0.098901
5,Cumbria,92,103,92,104,132,72,45,40,29,-0.684783
6,Derbyshire,171,179,176,174,178,123,130,126,122,-0.286550
7,Devon and Cornwall,116,126,120,148,120,106,106,86,63,-0.456897
8,Dorset,56,67,52,73,52,61,35,38,23,-0.589286
9,Durham,82,76,80,64,79,41,56,50,57,-0.304878


Dropping the last two columns

In [53]:
new_total_table = new_total_table.drop(["2022", "per_change_2014" ], axis=1)

In [55]:
new_total_table.head()

,pfa,2014,2015,2016,2017,2018,2019,2020,2021
0,Avon and Somerset,196,165,164,158,148,151,103,103
1,Bedfordshire,69,80,53,53,36,31,23,20
2,Cambridgeshire,91,89,112,115,116,89,78,47
3,Cheshire,169,181,167,172,176,149,123,117
4,Cleveland,91,78,108,152,140,98,55,103


In [56]:
old_total_table = pd.read_csv('data/processed/cust_sentences_total_table.csv')
old_total_table.head()

,pfa,2014,2015,2016,2017,2018,2019,2020,2021,per_change_2014
0,Avon and Somerset,196,165,164,158,148,151,103,103,-0.474490
1,Bedfordshire,69,80,53,53,36,31,23,20,-0.710145
2,Cambridgeshire,91,89,112,115,116,89,78,47,-0.483516
3,Cheshire,169,181,167,172,176,149,123,117,-0.307692
4,Cleveland,91,78,108,152,140,98,55,103,0.131868


In [57]:
old_total_table = old_total_table.drop("per_change_2014", axis=1)

In [58]:
old_total_table.head()

,pfa,2014,2015,2016,2017,2018,2019,2020,2021
0,Avon and Somerset,196,165,164,158,148,151,103,103
1,Bedfordshire,69,80,53,53,36,31,23,20
2,Cambridgeshire,91,89,112,115,116,89,78,47
3,Cheshire,169,181,167,172,176,149,123,117
4,Cleveland,91,78,108,152,140,98,55,103


In [59]:
old_total_table.compare(new_total_table)

2017       
     self  other
7   147.0  148.0
22  235.0  236.0

Same observation here of two entries from 2017 with a value of one higher—which is as expected.

In [65]:
new_total_table.iloc[[7, 22]]

,pfa,2014,2015,2016,2017,2018,2019,2020,2021
7,Devon and Cornwall,116,126,120,148,120,106,106,86
22,Merseyside,247,266,265,236,203,213,163,196


Right, all good. Re-running the consolidated code in one cell now.

In [ ]:
import src.data.utilities as utils
%load_ext autoreload
%autoreload 2

import pandas as pd

#Importing cleansed dataset
df = pd.read_csv('data/interim/PFA_2010-22_women_cust_comm_sus.csv')


## 1.SENTENCING OUTCOME FOR EACH PFA BY YEAR

#Grouping dataset and saving out
(
    df
    .groupby(['pfa', 'year', 'outcome'], as_index=False)['freq'].sum()
    .to_csv('data/processed/PFA_2010-22_women_sentencing_outcomes_FINAL.csv', index=False)
)


## 2. CUSTODIAL SENTENCES FOR EACH PFA BY OFFENCE TYPE

#Filtering cleansed dataset and grouping by PFA and offence group 
sentence_type = 'Immediate custody'
year = 2022

pfa_custody = (
    df
    .query("outcome == @sentence_type & year == @year")
    .groupby(['pfa', 'offence'], as_index=False)['freq'].sum()
)

#Using crosstab with normalize argument to calculate offence group proportions by PFA
pfa_custody_offences = pd.crosstab(index=pfa_custody['pfa'], columns=pfa_custody['offence'], values=pfa_custody['freq'], aggfunc=sum, normalize='index').round(3)


## 3.CUSTODIAL SENTENCE LENGTHS FOR EACH PFA BY YEAR
'''THIS PRODUCES THE DATA FOR FIGURE 1 IN THE PFA FACTSHEET'''

#Defining sentence_length categories—THESE HAVE BEEN UPDATED
less_6months = ["Up to and including 1 month", 
                "More than 1 month and up to and including 2 months",
                "More than 2 months and up to and including 3 months",
                "More than 3 months and up to 6 months"]

six_12_months = ["6 months",
                "More than 6 months and up to and including 9 months",
                "More than 9 months and up to 12 months"]

#Mapping sentence_len categories
def sentence_length_groups(sentence_len):
    if sentence_len in less_6months:
        return 'Less than 6 months'
    elif sentence_len in six_12_months:
        return '6 months and under 12 months'
    else:
        return 'Over 12 months'

#Filtering for custodial sentences and applying the map

pfa_custody_sentence_lengths = df.query('outcome == @sentence_type').copy()
pfa_custody_sentence_lengths['sentence_length'] = pfa_custody_sentence_lengths['sentence_length'].map(sentence_length_groups)

#Grouping dataset
pfa_custody_sentence_lengths = pfa_custody_sentence_lengths.groupby(['pfa', 'year', 'sentence_length'], as_index=False)['freq'].sum()

#Outputting to CSV
pfa_custody_sentence_lengths.to_csv('data/interim/PFA_2010-22_women_cust_sentence_length.csv', index=False)


## 4. CUSTODIAL SENTENCES FOR EACH PFA BY YEAR

'''THIS PRODUCES THREE DATASETS: 
    * TOTAL NUMBER OF WOMEN SENTENCED TO CUSTODY BY PFA; AND OF THOSE 
        * SENTENCED TO LESS THAN SIX MONTHS; AND
        * SENTENCED TO LESS THAN 12 MONTHS'''

#FILTERING DATA

#By year
filt = pfa_custody_sentence_lengths['year'] >= 2014
pfa_df_2014 = pfa_custody_sentence_lengths[filt].copy()

#By sentences of less than six months
filt = pfa_df_2014['sentence_length'] == "Less than 6 months"
lt_6m = pfa_df_2014[filt].copy()

#By sentences of less than 12 months
filt = pfa_df_2014['sentence_length'] != "Over 12 months"
lt_12m = pfa_df_2014[filt].copy()

#Defining new function for aggregating data and adding a percentage change column
def aggregate_sentences(df):
    new_df = pd.crosstab(index=df['pfa'], columns=df['year'],
                        values=df['freq'], aggfunc='sum')
    
    new_df = new_df.fillna(0.0).astype(int)
    new_df['per_change_2014'] = new_df.pct_change(axis='columns', periods=8).dropna(axis='columns')
    return new_df

#Using dictionary comprehension to run both DataFrames through the function
'''This returns a new dictionary df_dict with _table added to the keys. Values can be accessed using df_dict['key'] and DataFrame functionality is retained
See https://stackoverflow.com/questions/51845732/apply-a-function-to-multiple-dataframes-return-multiple-dfs-as-output'''

sentence_length_dict = {'cust_sentences_total':pfa_df_2014, 'cust_sentences_lt_6m':lt_6m, 'cust_sentences_lt_12m': lt_12m}
df_dict = {i+'_table': aggregate_sentences(sentence_length) for i, sentence_length in sentence_length_dict.items()}

#Outputting to CSV
#These are the final versions ready for formatting and publication
for i, df in df_dict.items():
    df.to_csv(f'data/processed/2022_{i}.csv')


,year,pfa,sex,age_group,offence,outcome,sentence_length,freq
0,2010,Avon and Somerset,Female,Young adults,Violence against the person,Community sentence,24:Not known,1
1,2010,Avon and Somerset,Female,Young adults,Drug offences,Community sentence,24:Not known,1
2,2010,Avon and Somerset,Female,Young adults,Violence against the person,Immediate custody,Life sentence,1
3,2010,Avon and Somerset,Female,Young adults,Violence against the person,Community sentence,24:Not known,1
4,2010,Avon and Somerset,Female,Young adults,Violence against the person,Suspended sentence,24:Not known,1


In [ ]:
def csnap(df, fn=lambda x: x.shape, msg=None):
    """ Custom Help function to print things in method chaining.
        Returns back the df to further use in chaining.
    """
    if msg:
        print(msg)
    display(fn(df))
    return df

In [ ]:
df.pipe(csnap)